# Introduction to PMUs and Synchrophasors

### Classic Definition of a Phasor

![](PMU1.png)


### Synchrophasors

![](PMU2.png)


### PMU

Device that estimates synchronised phasors, frequency, rate of change of frequency (ROCOF) from voltage and current samples and a time synchronised signal.

PMU may also perform other functions (e.g. fault recorder, e.g. calculation of power, power quality indices, etc.)

#### Phasor Estimation

![](PMU3.png)


The **first part** of modifiable parameters are as follows:

- f0 : Signal Frequency
- T0 : signal period
- N : samples per period
- fs : sampling frequency
- Nw : number of observation windows
- Nt : total number of samples

In [ ]:
f0 = 50;
T0 = 1/f0;
N = 10;
fs = f0*N;
Nw = 10;
Nt = N*Nw;

As the next step, based on the total number of samples collected, it is necessary to generate a time domain.

Time Domain Generation:

In [2]:
dt = 1/fs;
t = dt*[0:Nt-1];

Now, it is time to generate signal to be used in the estimation. 
The **second part** of modifiable parameters are as follows:

- amp : Amplitude
- phase: Phase Angle

In [4]:
amp = 50;
phase = 45;

x = amp*cos(2*pi*f0*t+phase); % x: signal over Nw cycles

### Synchrophasor Estimation

#### Non Recursive Method

![](PMU4.png)

Implementation of this method on Octave is illustrated below:


In [ ]:
X = zeros(1,Nw);
th = 2*pi/N;
X_list = num2cell(zeros(1,Nw));
col_num = 1;

for k = 1:Nw
  X = 0;
  for n = 0:N-1
    x_check = x(k+n);
    X = X + ((sqrt(2)/N)*(x(k+n)*exp(-1i*n*th))); % Phasor Calculation
    X_list(1,col_num) = {[X]};
    col_num = col_num + 1;
  endfor
endfor

### Total Vector Error (TVE)

![](PMU5.png)

Implementation of this method on Octave is illustrated below:


In [ ]:
phasor_actual = cell2mat(X_list(1,1));
phasor_estimate = cell2mat(X_list(1,2));

real_diff = real(phasor_estimate) - real(phasor_actual);
imag_diff = imag(phasor_estimate) - imag(phasor_actual);
TVE = sqrt(((real_diff)^2 + (imag_diff)^2)/((real(phasor_actual)^2) + (imag(phasor_actual)^2)));

#### Graphical Representation of TVE

In [ ]:
line([0 real(phasor_actual)],[0 imag(phasor_actual)], "linestyle", "-", "color", "g")
line([0 real(phasor_estimate)],[0 imag(phasor_estimate)], "linestyle", "-", "color", "r")
line([real(phasor_estimate) real(phasor_actual)], [imag(phasor_estimate) imag(phasor_actual)], "linestyle", "--", "color", "k")
title("Total Vector Error (TVE) in Recursive Phasor Calculations")
legend('Actual Phasor','Estimated Phasor','TVE', 'location', 'eastoutside');
disp ("Total Vector Error is:"), disp (TVE)